# Plotting the Region of Attraction and its Estimations

### imports

In [ ]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import time as t

# Local Imports
#from simple_pendulum.controllers.lqr.lqr import lqr
from simple_pendulum.controllers.lqr.lqr_controller import LQRController
from simple_pendulum.model.pendulum_plant import PendulumPlant

#from simple_pendulum.controllers.lqr.analytic_roa_estimation.lqr_controller_roa import LQRControllerCapped
from simple_pendulum.controllers.lqr.analytic_roa_estimation.najafi_oracle import najafi_oracle
from simple_pendulum.controllers.lqr.analytic_roa_estimation.analytic_roa import analytic_roa


USE_DATA_FROM_PAPER = True

data_folder = "sim_data"
if USE_DATA_FROM_PAPER:
    data_folder = "sim_data_from_paper"

### functions that estimate the RoA

In [ ]:
def analyse(table, predictions = [], verbose = True):
    # table[:,2] 'converges', table[:,3] 'violates'
    index = range(len(table))
    diverges = []
    converges = []
    behaves = []   
    for i in index:
        if table[i,2]:
            if not table[i,3]:
                behaves.append(table[i,0:2])
            else:
                converges.append(table[i,0:2])
        else:
            diverges.append(table[i,0:2])
    out = [diverges, converges, behaves]
    
    if verbose:
        print('\033[1m' + "Description of Indices:" + '\033[0m')
        print()
        print("\033[4m" + "general" + '\033[0m')
        print("0: diverge")
        print("1: converge")
        print("2: converge + no violation")
        print()
        n=3
    
    for test in predictions:
        
        if verbose:
            print("\033[4m" + test[0] + '\033[0m')
            print(str(n) + ": positive")
            print(str(n+1) + ": false positive")
            print(str(n+2) + ": negative")
            print(str(n+3) + ": false negative")
            print()
            n += 4
        
        pos = []
        falsepos = []
        neg = []
        falseneg = []
        for i in index:
            q = table[i,0:2]
            if test[1](q):
                if table[i,2]:
                    pos.append(q)
                else:
                    falsepos.append(q)
            else:
                if table[i,2]:
                    falseneg.append(q)
                else:
                    neg.append(q)
        out.extend([pos,falsepos,neg,falseneg])
    return out

In [ ]:
def getSets(data_path, verbose = True):
    params = np.genfromtxt(data_path + "/parameters.csv", delimiter = ",")
    table = np.genfromtxt(data_path + "/table.csv", delimiter = ",")
    
    mass = params[0]
    length = params[1]
    torque_limit = params[2]
    damping = params[3]
    coulomb_fric = params[4]
    gravity = params[5]
    Q_11 = params[6]
    Q_22 = params[7]
    R = params[8]
    N = int(params[9])
    inertia = mass*length*length
    
    t0 = t.time()
    analytic_estimator = analytic_roa(mass=mass,
                                      length=length,
                                      damping=damping,
                                      gravity=gravity,
                                      torque_limit=torque_limit,
                                      Q_11 = Q_11,
                                      Q_22 = Q_22,
                                      R = R)
    
    analytic = analytic_estimator.satisfies_theory
    analytic2 = analytic_estimator.satisfies_theory_full
    

    t1 = t.time()
    pendulum = PendulumPlant(mass=mass,
                            length=length,
                            damping=damping,
                            gravity=gravity,
                            coulomb_fric=coulomb_fric,
                            inertia=inertia,
                            torque_limit=torque_limit)

    controller = LQRController(mass=mass,
                           length=length,
                           damping=damping,
                           gravity=gravity,
                           torque_limit=torque_limit,
                           Q = np.diag((Q_11,Q_22)),
                           R = np.array([[R]]))
    controller.set_clip()

    orac = najafi_oracle(pendulum,controller)
    najafi = orac.query
    t2 = t.time()
    
    return analyse(table, predictions = [["analytic", analytic], ["najafi", najafi]], verbose = verbose), t1-t0, t2-t1

def getSetHeuristic(data_path, verbose = True):
    params = np.genfromtxt(data_path + "/parameters.csv", delimiter = ",")
    table = np.genfromtxt(data_path + "/table.csv", delimiter = ",")
    
    mass = params[0]
    length = params[1]
    torque_limit = params[2]
    damping = params[3]
    coulomb_fric = params[4]
    gravity = params[5]
    Q_11 = params[6]
    Q_22 = params[7]
    R = params[8]
    N = int(params[9])
    inertia = mass*length*length
    
    t0 = t.time()
    analytic_estimator = analytic_roa(mass=mass,
                                      length=length,
                                      damping=damping,
                                      gravity=gravity,
                                      torque_limit=torque_limit,
                                      Q_11 = Q_11,
                                      Q_22 = Q_22,
                                      R = R)
    
    analytic = analytic_estimator.satisfies_theory
    without_heuristic = analytic_estimator.satisfies_theory_full
    t1 = t.time()

    return analyse(table, predictions = [["analytic", analytic], ["without heuristic", without_heuristic]], verbose = verbose), t1-t0, 0

### functions for drawing the plots

In [ ]:
def draw(temp, savename = None, boundaries = [0,2*np.pi,-10,10], xticks = [0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi], xlabels = ['$-\pi$', "$-\pi/2$","$0$","$\pi/2$","$\pi$"]):
    plt.style.use(['science', 'ieee'])
    fig,ax = plt.subplots(figsize=(2,1.3333333333))
    
    s=.3
    try:
        X,Y = zip(*temp[1])
        plt.scatter(X,Y, c = "gray", s=s, label=r'$\mathcal{S}$')
    except: pass
    try:
        X,Y = zip(*temp[2])
        plt.scatter(X,Y, c = "black", s=s, label=r'$\tilde{\mathcal{S}}$')
    except: pass
    try:
        X,Y = zip(*temp[7])
        plt.scatter(X,Y, c = "#ffd700", s=s, label=r'$\mathcal{S}_\text{Lyapunov}$') 
    except: pass
    try:
        X,Y = zip(*temp[3])
        plt.scatter(X,Y, c = "#0057b7", s=s, label=r'$\mathcal{S}_\text{analytic}$')   
    except: pass
    
    try:
        X,Y = zip(*temp[8])
        plt.scatter(X,Y, c = "orange", s=s, label=r'$\mathcal{S}_\text{problematic}$')
    except: pass
    
    ax.axis(boundaries)
    
    
    ax.set_xticks(xticks)
    ax.set_xticklabels(xlabels)
    
    
    P1 = mpatches.Patch(color = "gray", label=r'$\mathcal{S}$')
    P2 = mpatches.Patch(color = "black", label=r'$\tilde{\mathcal{S}}$')
    P3 = mpatches.Patch(color = "#ffd700", label=r'$\mathcal{S}_\text{Lyapunov}$')
    P4 = mpatches.Patch(color = "#0057b7", label=r'$\mathcal{S}_\text{analytic}$')
    leg = ax.legend(handles=[P1,P2,P3,P4], loc = 1, labelspacing = 0, frameon = True, edgecolor = "black", prop={'size': 6})
    leg.get_frame().set_linewidth(0.5)
    leg.get_frame().set_alpha(None)
    leg.get_frame().set_facecolor((1, 1, 1, .5))
    
    plt.xlabel('angle [rad]')
    plt.ylabel('angular velocity [rad/s]')
    if not savename == None:
        plt.savefig(savename + '.png')
    else:
        plt.show()
        
def drawHeuristic(temp, savename = None, show = True, boundaries = [0,2*np.pi,-10,10], xticks = [0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi], xlabels = ['$-\pi$', "$-\pi/2$","$0$","$\pi/2$","$\pi$"]):
    plt.style.use(['science', 'ieee'])
    fig,ax = plt.subplots(figsize=(3.3, 2.2))
    plt.xlabel('angle [rad]')
    plt.ylabel('angular velocity [rad/s]')
    
    s=.3
    try:
        X,Y = zip(*temp[1])
        plt.scatter(X,Y, c = "gray", s=s, label=r'$\mathcal{S}$')
    except: pass
    try:
        X,Y = zip(*temp[2])
        plt.scatter(X,Y, c = "black", s=s, label=r'$\tilde{\mathcal{S}}$')
    except: pass
    try:
        X,Y = zip(*temp[7])
        plt.scatter(X,Y, c = "tab:red", s=s, label=r'$\mathcal{S}_\text{unbound}$') 
    except: pass
    try:
        X,Y = zip(*temp[3])
        plt.scatter(X,Y, c = "#0057b7", s=s, label=r'$\mathcal{S}_\text{analytic}$')
    except: pass
    try:
        X,Y = zip(*temp[8])
        plt.scatter(X,Y, c = "tab:red", s=s)
    except: pass
    
    ax.axis(boundaries)
    
    
    ax.set_xticks(xticks)
    ax.set_xticklabels(xlabels)
    P1 = mpatches.Patch(color = "gray", label=r'$\mathcal{S}$')
    P2 = mpatches.Patch(color = "black", label=r'$\tilde{\mathcal{S}}$')
    P3 = mpatches.Patch(color = "tab:red", label=r'$\mathcal{S}_\text{unbound}$')
    P4 = mpatches.Patch(color = "#0057b7", label=r'$\mathcal{S}_\text{analytic}$')
    leg = ax.legend(handles=[P1,P2,P3,P4], loc = 1, labelspacing = 0, frameon = True, edgecolor = "black")
    leg.get_frame().set_linewidth(0.5)
    leg.get_frame().set_alpha(None)
    leg.get_frame().set_facecolor((1, 1, 1, .5))
    
    plt.xlabel('angle [rad]')
    plt.ylabel('angular velocity [rad/s]')
    if not savename == None:
        plt.savefig(savename + '.png')
    else:
        plt.show()

### drawing the comparison between the analytic estimation with and without the heuristic bound

In [ ]:
if not os.path.isdir("figures"):
    os.makedirs("figures")

In [ ]:
run, boundaries, xticks, xlabels = ["long_4", [0,2*np.pi,-9,9], [0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi], ['$-\pi$', "$-\pi/2$","$0$","$\pi/2$","$\pi$"]]
data_path = os.path.join(os.getcwd(), data_folder, "parameters_and_tables", run)
temp, t_ana, _ = getSetHeuristic(data_path, verbose = False)
drawHeuristic(temp, savename = 'figures/heuristic', show = False, boundaries = boundaries, xticks=xticks, xlabels=xlabels)

### drawing the comparison between the analytic and the lyapunov based approach

In [ ]:
runs = ["normal_2", "normal_4", "normal_8",
        "long_2", "long_4", "long_8", 
        "short_2", "short_4", "short_8"]
runss = [["normal_2", [0,2*np.pi,-10,10], [0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi], ['$-\pi$', "$-\pi/2$","$0$","$\pi/2$","$\pi$"]], 
         ["normal_4", [np.pi/2,3*np.pi/2,-7,7], [np.pi/2, 3*np.pi/4, np.pi, 5*np.pi/4, 3*np.pi/2], ["$-\pi/2$","$-\pi/4$","$0$","$\pi/4$","$\pi/2$"]], 
         ["normal_8", [3*np.pi/4,5*np.pi/4,-4,4], [3*np.pi/4, np.pi*(1-1/8), np.pi, np.pi*(1+1/8), 5*np.pi/4], ["$-\pi/4$","$-\pi/8$","$0$","$-\pi/8$","$\pi/4$"]],
         ["long_2", [0,2*np.pi,-7,7], [0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi], ['$-\pi$', "$-\pi/2$","$0$","$\pi/2$","$\pi$"]], 
         ["long_4", [np.pi/4,7*np.pi/4,-6,6], [np.pi/2, 3*np.pi/4, np.pi, 5*np.pi/4, 3*np.pi/2], ["$-\pi/2$","$-\pi/4$","$0$","$\pi/4$","$\pi/2$"]], 
         ["long_8", [3*np.pi/8,13*np.pi/8,-5,5], [np.pi/2, 3*np.pi/4, np.pi, 5*np.pi/4, 3*np.pi/2], ["$-\pi/2$","$-\pi/4$","$0$","$\pi/4$","$\pi/2$"]],
         ["short_2", [np.pi/2,3*np.pi/2,-8,8], [np.pi/2, 3*np.pi/4, np.pi, 5*np.pi/4, 3*np.pi/2], ["$-\pi/2$","$-\pi/4$","$0$","$\pi/4$","$\pi/2$"]], 
         ["short_4", [3*np.pi/4,5*np.pi/4,-5,5], [3*np.pi/4, np.pi*(1-1/8), np.pi, np.pi*(1+1/8), 5*np.pi/4], ["$-\pi/4$","$-\pi/8$","$0$","$-\pi/8$","$\pi/4$"]], 
         ["short_8", [7*np.pi/8,9*np.pi/8,-3,3], [np.pi*(1-1/8), np.pi, np.pi*(1+1/8)], ["$-\pi/8$","$0$","$-\pi/8$"]]
        ]

In [ ]:
volume = {}
times = {}
for (run, boundaries, xticks, xlabels) in runss:
    data_path = os.path.join(os.getcwd(), data_folder, "parameters_and_tables", run)
    temp, t_ana, t_naj= getSets(data_path, verbose = False)
    times[run]=[t_ana, t_naj]
    volume[run]=[len(temp[3]), len(temp[7])]
    draw(temp, savename = 'figures/'+run, boundaries = boundaries, xticks=xticks, xlabels=xlabels)
    print(run + " completed")

### comparison of volumes and computation times

In [ ]:
volume

In [ ]:
times

In [ ]:
for run in runs:
    print(run +": "+ str(volume[run][0]/volume[run][1]))

In [ ]:
t_mean_ana = 0
t_mean_naj = 0
for run in runs:
    t_mean_ana += times[run][0]
    t_mean_naj += times[run][1]
    print(run +": "+ str(times[run][0]/times[run][1]))
t_mean_ana /= len(runs)
t_mean_naj /= len(runs)
print("analytic mean time: " + str(t_mean_ana))
print("najafi mean time: " + str(t_mean_naj))